# PROJEKT1 IOwADC 

### Karol Widuch + Konrad Kalita
#### Breast Cancer + regresja logistyczna + precyzja

imports

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn import metrics
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(
    'breast-cancer.data', sep=',',
    names=['Class', 'age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps', 'deg-malig', 'breast', 'breast-quad', 'irradiat']
)

df.head()

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


Wartości kolumn:
   1. Class: 
           no-recurrence-events, recurrence-events
   2. age: 
           10-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80-89, 90-99.
   3. menopause: 
           lt40, ge40, premeno.
   4. tumor-size: 
          0-4, 5-9, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40-44, 45-49, 50-54, 55-59.
   5. inv-nodes: 
           0-2, 3-5, 6-8, 9-11, 12-14, 15-17, 18-20, 21-23, 24-26, 27-29, 30-32, 33-35, 36-39.
   6. node-caps: 
           yes, no.
   7. deg-malig: 
           1, 2, 3.
   8. breast: 
           left, right.
   9. breast-quad: 
           left-up, left-low, right-up,	right-low, central.
   10. irradiat:
           yes, no.

In [3]:
for col in df.columns:
    print(df[col].value_counts())
    print("##############################")

no-recurrence-events    201
recurrence-events        85
Name: Class, dtype: int64
##############################
50-59    96
40-49    90
60-69    57
30-39    36
70-79     6
20-29     1
Name: age, dtype: int64
##############################
premeno    150
ge40       129
lt40         7
Name: menopause, dtype: int64
##############################
30-34    60
25-29    54
20-24    50
15-19    30
10-14    28
40-44    22
35-39    19
50-54     8
0-4       8
5-9       4
45-49     3
Name: tumor-size, dtype: int64
##############################
0-2      213
3-5       36
6-8       17
9-11      10
15-17      6
12-14      3
24-26      1
Name: inv-nodes, dtype: int64
##############################
no     222
yes     56
?        8
Name: node-caps, dtype: int64
##############################
2    130
3     85
1     71
Name: deg-malig, dtype: int64
##############################
left     152
right    134
Name: breast, dtype: int64
##############################
left_low     110
left_up       97
right_up

In [4]:
df_dummies = pd.get_dummies(df, columns=df.select_dtypes(['object']).columns)
df_dummies.head()

,deg-malig,Class_no-recurrence-events,Class_recurrence-events,age_20-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-79,menopause_ge40,...,breast_left,breast_right,breast-quad_?,breast-quad_central,breast-quad_left_low,breast-quad_left_up,breast-quad_right_low,breast-quad_right_up,irradiat_no,irradiat_yes
0,3,1,0,0,1,0,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
1,2,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,1,1,0
2,2,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,1,0
3,2,1,0,0,0,0,0,1,0,1,...,0,1,0,0,0,1,0,0,1,0
4,2,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0


In [5]:
df_dummies.drop("Class_no-recurrence-events", axis=1, inplace=True)
df_dummies.drop("breast_left", axis=1, inplace=True)
df_dummies.drop("irradiat_yes", axis=1, inplace=True)

df_dummies.shape

(286, 40)

In [6]:
df_dummies.describe().round(2)

,deg-malig,Class_recurrence-events,age_20-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-79,menopause_ge40,menopause_lt40,...,node-caps_no,node-caps_yes,breast_right,breast-quad_?,breast-quad_central,breast-quad_left_low,breast-quad_left_up,breast-quad_right_low,breast-quad_right_up,irradiat_no
count,286.00,286.00,286.00,286.00,286.00,286.00,286.0,286.00,286.00,286.00,...,286.00,286.0,286.00,286.00,286.00,286.00,286.00,286.00,286.00,286.00
mean,2.05,0.30,0.00,0.13,0.31,0.34,0.2,0.02,0.45,0.02,...,0.78,0.2,0.47,0.00,0.07,0.38,0.34,0.08,0.12,0.76
std,0.74,0.46,0.06,0.33,0.47,0.47,0.4,0.14,0.50,0.15,...,0.42,0.4,0.50,0.06,0.26,0.49,0.47,0.28,0.32,0.43
min,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,...,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,2.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,...,1.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
50%,2.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,...,1.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00
75%,3.00,1.00,0.00,0.00,1.00,1.00,0.0,0.00,1.00,0.00,...,1.00,0.0,1.00,0.00,0.00,1.00,1.00,0.00,0.00,1.00
max,3.00,1.00,1.00,1.00,1.00,1.00,1.0,1.00,1.00,1.00,...,1.00,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


In [7]:
y = df_dummies['Class_recurrence-events']
X = df_dummies.drop('Class_recurrence-events', axis=1)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, shuffle=True)

In [9]:
lr = linear_model.LogisticRegression()
lr.fit(X_train, y_train)

lr.score(X_test, y_test)

0.7241379310344828

In [10]:
scores = cross_val_score(lr, X_test, y_test, cv=10)

for i, score in enumerate(scores):
    print(f"num: {i} \t result: {score}")

print(f"\nmax: \t{scores.max()}")
print(f"min: \t{scores.min()}")


num: 0 	 result: 0.8333333333333334
num: 1 	 result: 0.6666666666666666
num: 2 	 result: 0.8333333333333334
num: 3 	 result: 0.5
num: 4 	 result: 0.6666666666666666
num: 5 	 result: 0.8333333333333334
num: 6 	 result: 0.6666666666666666
num: 7 	 result: 0.5
num: 8 	 result: 0.6
num: 9 	 result: 0.8

max: 	0.8333333333333334
min: 	0.5


In [17]:
kf = KFold(n_splits=10, shuffle=True)

for i, [train_idx, valid_idx] in enumerate(kf.split(X)):
    
    X_t_f, X_v_f = X.iloc[train_idx], X.iloc[valid_idx]
    y_t_f, y_v_f = y.iloc[train_idx], y.iloc[valid_idx]
    
    lr.fit(X_t_f, y_t_f)
    print(f"{i+1}: \t{lr.score(X_test, y_test)}")

1: 	0.7758620689655172
2: 	0.7758620689655172
3: 	0.7413793103448276
4: 	0.7931034482758621
5: 	0.7758620689655172
6: 	0.7931034482758621
7: 	0.7931034482758621
8: 	0.7931034482758621
9: 	0.7931034482758621
10: 	0.7758620689655172
